L2-Regularization Method on Link Prediction Task of Cora Dataset
--------------------------

### All libraries we need

In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import os.path as osp
from os.path import abspath, dirname
import shutil
import numpy as np
import copy
import time
import statistics as stat
import psutil
import itertools
import tracemalloc
import gc
import dill
import csv
import datatable as dt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from typing import Dict, List, Union, Tuple, Optional, Any
import statistics as stat
import pprint
from argparse import ArgumentParser



from utils.loss_functions import JointLoss
from utils.model_plot import save_auc_plot, save_loss_plot
from utils.model_utils import GAEWrapper
from utils.utils import set_dirs, set_seed
from utils.utils import get_runtime_and_model_config, print_config


import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader,IterableDataset
import torch_geometric.transforms as T
from torch_geometric.seed import seed_everything as th_seed
from torch_geometric.utils import dropout_adj
from torch_geometric.data import Data
from torch.utils.data import DataLoader, Dataset
from torch_geometric.datasets import Planetoid, WebKB, WikipediaNetwork
from torch_geometric.nn import GAE, VGAE, GCNConv





from utils.utils import set_dirs, update_config_with_model_dims
from utils.loss_functions import  JointLoss

torch.autograd.set_detect_anomaly(True)

### Regularization Rate
#### Regularization rates range from the following numbers:

In [2]:

0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1e2, 1e3, 1e6
;

''

### Functions for Measuring criterias

In [3]:


def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for name, param in model.autoencoder.gae.encoder.linear1.named_parameters():
     
    

        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements


# Function to get CPU usage
def get_cpu_usage():
    return psutil.cpu_percent(interval=1)



# Function to approximate power consumption (Assume some average power usage per CPU percentage point)
def estimate_power_usage(cpu_usage):
    base_power_usage = 10  # Assumed base power usage in watts
    power_per_percent = 0.5  # Assumed additional watts per CPU usage percent
    return base_power_usage + (power_per_percent * cpu_usage)

# The model size based on the number of parameters
def get_model_size_param(model: nn.Module, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width

### Start loading data

#### functions for data loaders.

In [4]:
#In this cell there some class and function for loading dataset and preprocessing

class GraphLoader:
    """
    Data loader class for graph data.
    """

    def __init__(self, config: Dict[str, Any], dataset_name: str, kwargs: Dict[str, Any] = {}) -> None:
        """
        Initializes the GraphLoader.

        Parameters
        ----------
        config : Dict[str, Any]
            Dictionary containing options and arguments.
        dataset_name : str
            Name of the dataset to load.
        kwargs : Dict[str, Any], optional
            Dictionary for additional parameters if needed, by default {}.
        """
        # Get config
        self.config = config
        # Set the seed
        th_seed(config["seed"])
        # Set the paths
        paths = config["paths"]
        # data > dataset_name
        file_path = os.path.join(paths["data"], dataset_name)
        # Get the datasets
        self.train_data, self.validation_data, self.test_data = self.get_dataset(dataset_name, file_path)        
        

    def get_dataset(self, dataset_name: str, file_path: str) -> Tuple[Data, Data, Data]:
        """
        Returns the training, validation, and test datasets.

        Parameters
        ----------
        dataset_name : str
            Name of the dataset to load.
        file_path : str
            Path to the dataset.

        Returns
        -------
        Tuple[Data, Data, Data]
            Training, validation, and test datasets.
        """

        # Initialize Graph dataset class
        graph_dataset = GraphDataset(self.config, datadir=file_path, dataset_name=dataset_name)
        
        # Load Training, Validation, Test datasets
        train_data, val_data, test_data = graph_dataset._load_data()
        
        # Generate static subgraphs from training set
        train_data = self.generate_subgraphs(train_data)
  
        # Return
        return train_data, val_data, test_data
    
    
    def generate_subgraphs(self, train_data: Data) -> List[Data]:
        """
        Generates subgraphs from the training data.

        Parameters
        ----------
        train_data : Data
            Training data containing the graph.

        Returns
        -------
        List[Data]
            List of subgraphs generated from the training data.
        """
        # Initialize list to hold subgraphs
        subgraphs = [train_data]

        # Check if we are generating subgraphs from the graph. If False, we are in standard GAE mode
        if self.config["n_subgraphs"] > 1:
                
            # Generate subgraphs
            for i in range(self.config["n_subgraphs"]):
                
                # Change random seed
                th_seed(i)
                
                partition = 1.0/(self.config["n_subgraphs"]-i)
                
                # For the last subgraph, get 95% of the remaining graph. if num_val=1.0, RandomLinkSplit will raise error
                if partition == 1.0:
                    partition = 0.95
                    
                random_link_split = T.RandomLinkSplit(num_val=partition, 
                                                      num_test=0, 
                                                      is_undirected=True, 
                                                      split_labels=True, 
                                                      add_negative_train_samples=False)

                # get a subgraph from training data
                train_data, train_subgraph, _ = random_link_split(train_data)
                
                # Make sure that we are using only the nodes within the subgraph by overwriting the edge index 
                # with positive edge index + positive edge index reversed in direction (to make it undirected)
                pos_swapped = train_subgraph.pos_edge_label_index[[1,0],:] 
                train_subgraph.edge_index = torch.cat((train_subgraph.pos_edge_label_index, pos_swapped), dim=1)
                
                # Remove negative edge attributes. We want to sample negative samples during training
                # Masks are also not needed
                if hasattr(train_subgraph, "neg_edge_label_index"):
                    delattr(train_subgraph, "neg_edge_label_index")
                    delattr(train_subgraph, "neg_edge_label")
                    
                if hasattr(train_subgraph, "train_mask"):
                    delattr(train_subgraph, "train_mask")
                    delattr(train_subgraph, "val_mask")
                    delattr(train_subgraph, "test_mask")


                # store the sampled subgraph
                subgraphs = [train_subgraph] + subgraphs
                       
        # Change random seed back to original
        th_seed(self.config["seed"])
        
        # Return all subgraphs and original larger graph
        return subgraphs

    
def get_transform(options):
    """Splits data to train, validation and test, and moves them to the device"""
    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToDevice(options["device"]),
        T.RandomLinkSplit(num_val=0.05, 
                          num_test=0.15, 
                          is_undirected=True,
                          split_labels=True, 
                          add_negative_train_samples=False),
        ])
        
    return transform


class GraphDataset:
    """
    Dataset class for graph data format.
    """

    def __init__(self, config: Dict[str, Any], datadir: str, dataset_name: str) -> None:
        """
        Initializes the GraphDataset.

        Parameters
        ----------
        config : Dict[str, Any]
            Dictionary containing options and arguments.
        datadir : str
            The path to the data directory.
        dataset_name : str
            Name of the dataset to load.
        """
        self.config = config
        self.paths = config["paths"]
        self.dataset_name = dataset_name
        self.data_path = os.path.join(self.paths["data"], 'Planetoid')
        self.transform = get_transform(config)

        
    def _load_data(self) -> Tuple[Data, Data, Data]:
        """
        Loads one of many available datasets and returns features and labels.

        Returns
        -------
        Tuple[Data, Data, Data]
            Training, validation, and test datasets.
        """
        if self.dataset_name.lower() in ['cora', 'citeseer', 'pubmed']:
            # Get the dataset
            dataset = Planetoid(self.data_path, self.dataset_name, split="random", transform = self.transform)
        elif  self.dataset_name.lower() in ['chameleon']:
            # Get the dataset
            dataset = WikipediaNetwork(root=self.data_path, name=self.dataset_name, transform = self.transform)
        elif  self.dataset_name.lower() in ["cornell", "texas", "wisconsin"]:
            # Get the dataset
            dataset = WebKB(root=self.data_path, name=self.dataset_name, transform = self.transform) 
        else:
            print(f"Given dataset name is not found. Check for typos, or missing condition ")
            exit()
            
        # Data splits
        train_data, val_data, test_data = dataset[0]
        
        # Return
        return train_data, val_data, test_data


### Loads arguments and configuration for GNN-based encoder used in NESS.

In [5]:

def get_arguments():
    # Initialize parser
    parser = ArgumentParser()
    # Dataset can be provided via command line
    parser.add_argument("-d", "--dataset", type=str, default="cora")
    # Encoder type
    parser.add_argument("-gnn", "--gnn", type=str, default="GNAE")
    # Random seed
    parser.add_argument("-seed", "--seed", type=int, default=57)
    # Whether to use contrastive loss
    parser.add_argument("-cl", "--cl", type=bool, default=False)
    # Whether to add noise to input
    parser.add_argument("-an", "--an", type=bool, default=True)
    # Whether to use GPU.
    parser.add_argument("-g", "--gpu", dest='gpu', action='store_true', 
                        help='Used to assign GPU as the device, assuming that GPU is available')
    
    parser.add_argument("-ng", "--no_gpu", dest='gpu', action='store_false', 
                        help='Used to assign CPU as the device')
    parser.set_defaults(gpu=True)
    
    # GPU device number as in "cuda:0". Defaul is 0.
    parser.add_argument("-dn", "--device_number", type=str, default='0', 
                        help='Defines which GPU to use. It is 0 by default')
    
    ### Set parameter for L2-Regularization and Pruning ##****************
    parser.add_argument("-is_reg", "--is_reg", type=bool, default=False)
    parser.add_argument("-reg", "--l2_reg", type=float, default='0', 
            help='Defines which GPU to use. It is 0 by default')

    parser.add_argument("-is_pruned", "--is_pruned", type=bool, default=False)
    parser.add_argument("-sparsity", "--sparsity", type=float, default='0', 
            help='Defines which GPU to use. It is 0 by default')
            
    
 
    
    
    # Experiment number
    parser.add_argument("-ex", "--experiment", type=int, default=1)
    # Load model saved at specific epoch
    parser.add_argument("-m", "--model_at_epoch", type=int, default=None)
    
    # Return parser arguments along with the unknown ones
    args, unknown = parser.parse_known_args()
    return args


def get_config(args):
    # Load runtime config from config folder: ./config/
    config = get_runtime_and_model_config(args)
    # Define which device to use: GPU or CPU
    config["device"] = torch.device('cuda:'+args.device_number if torch.cuda.is_available() and args.gpu else 'cpu')
    # Model at specific epoch
    config["model_at_epoch"] = args.model_at_epoch
    # Indicate which device is being used
    config["l2_reg"] = args.l2_reg

    config["is_reg"]= args.is_reg
    


    print(f"Device being used is {config['device']}")
    # Return
    return config

def print_config_summary(config, args=None):
    """Prints out summary of options and arguments used"""
    # Summarize config on the screen as a sanity check
    print(100 * "=")
    print(f"Here is the configuration being used:\n")
    print_config(config)
    print(100 * "=")
    if args is not None:
        print(f"Arguments being used:\n")
        print_config(args)
        print(100 * "=")


In [6]:
# Get parser / command line arguments
args = get_arguments()
# Get configuration file
config = get_config(args)


# By default, we are using the name of the dataset. This can be customized.
config["experiment"] = config["dataset"]

# File name to use when saving results as csv. This can be customized
config["file_name"] = config["experiment"] + "_sub" + str(config["n_subgraphs"]) + '_seed' + str(config["seed"])

Device being used is cpu


In [7]:
# Summarize config and arguments on the screen as a sanity check
print_config_summary(config, args)


Here is the configuration being used:

+-------------------+----------------------------------------------+
|     Parameter     |                    Value                     |
+===================+==============================================+
| Add noise         | True                                         |
+-------------------+----------------------------------------------+
| Aggregation       | mean                                         |
+-------------------+----------------------------------------------+
| Batch size        | 128                                          |
+-------------------+----------------------------------------------+
| Contrastive loss  | False                                        |
+-------------------+----------------------------------------------+
| Cosine similarity | False                                        |
+-------------------+----------------------------------------------+
| Dataset           | cora                                      

### NESS

#### NESS class, the framework used to learn node embeddings from static subgraphs.

In [8]:
### It is not realll
import csv
import gc
import itertools
import os
import time

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torch_geometric.transforms as T
from torch_geometric.seed import seed_everything as th_seed
from torch_geometric.utils import dropout_adj
from torch_geometric.data import Data

from tqdm import tqdm
from typing import Dict, List, Union, Tuple, Optional

from utils.loss_functions import JointLoss
from utils.model_plot import save_auc_plot, save_loss_plot
from utils.model_utils import GAEWrapper
from utils.utils import set_dirs, set_seed

torch.autograd.set_detect_anomaly(True)

class NESS:
    """
    Model: Trains a Graph Autoencoder with a Projection network, using NESS framework.
    """

    def __init__(self, config: Dict):
        """Initializes the NESS class.

        Parameters
        ----------
        config : dict
            Configuration dictionary with parameters for training a Graph Autoencoder 
            using NESS framework.
        """
        # Get config
        self.config = config
        # Set L2-Regularization Parameters******************
        self.is_reg=config["is_reg"]
        self.l2_reg=config["l2_reg"]
    
        if self.l2_reg!= 0:
            self.is_reg=True
         
        print(f" Is the approach is regularization:{self.is_reg}")
        print(f"Regrularization rate is:{self.l2_reg}")
        
        
        # Define which device to use: GPU, or CPU
        self.device = config["device"]
        # Create empty lists and dictionary
        self.model_dict, self.summary = {}, {}
        # Set random seed
        set_seed(self.config)
        # Set paths for results and initialize some arrays to collect data during training
        self._set_paths()
        # Set directories i.e. create ones that are missing.
        set_dirs(self.config)
        # ------Network---------
        # Instantiate networks
        print("Building the models for training and evaluation in NESS framework...")
        # Set Autoencoders i.e. setting loss, optimizer, and device assignment (GPU, or CPU)
        self.set_autoencoder()
        # Set scheduler (its use is optional)
        self._set_scheduler()
        # Print out model architecture
        self.print_model_summary()
        

        
    def set_autoencoder(self) -> None:
        """Sets up the autoencoder model, optimizer, and loss.
        
        This function is responsible for initializing the Graph Autoencoder, setting up 
        the optimizer, and defining the joint loss.
        """   
        # Instantiate the model for the text Autoencoder
        self.autoencoder = GAEWrapper(self.config)
        # Add the model and its name to a list to save, and load in the future
        self.model_dict.update({"autoencoder": self.autoencoder})
        # Assign autoencoder to a device
        for _, model in self.model_dict.items():
            model.to(self.device)
        # Get model parameters
        parameters = [model.parameters() for _, model in self.model_dict.items()]
        # Joint loss including contrastive, reconstruction and distance losses
        
        #for param in   parameters:
            #l2_reg += torch.norm(param)
    
        # Combine the loss function with L2 regularization
        #loss += (l2 * l2_reg)
    
        
        self.joint_loss = None if self.config["dataset"][:4] == "ogbl" else JointLoss(self.config)
        # Set optimizer for autoencoder
        
        # Set optimizer for autoencoder with L2 regularization
        self.optimizer_ae = self._adam(parameters, lr=self.config["learning_rate"], weight_decay=0)
        ##  self.optimizer_ae = self._adam(parameters, lr=self.config["learning_rate"])
        
        # Add items to summary to be used for reporting later
        self.summary.update({"recon_loss": []})

    def set_parallelism(self, model) -> None:
        """Sets up parallelism in training if multiple GPUs are available.

        Parameters
        ----------
        model : torch.nn.Module
            The model for which the parallelism is to be set.

        Returns
        -------
        model : torch.nn.Module
            The input model wrapped in DataParallel if multiple GPUs are available.
        """
        # If we are using GPU, and if there are multiple GPUs, parallelize training
        if torch.cuda.is_available() and torch.cuda.device_count() > 1:
            print(torch.cuda.device_count(), " GPUs will be used!")
            model = torch.nn.DataParallel(model)
        return model

    def fit(self, data_loader: DataLoader) -> None:
        """Fits model to the data.

        Parameters
        ----------
        data_loader : DataLoader
            The DataLoader object that provides the data.
        """
        
        # Get data loaders
        train_data = data_loader.train_data
        validation_data = data_loader.validation_data
        test_data = data_loader.test_data

        # Placeholders to record losses per batch
        self.metrics = {"tloss_e": [], "vloss_e": [], "rloss_e": [], "zloss_e": [], "val_auc": [], "tr_auc": []}
        self.val_auc = "NA"
        self.tr_auc = "NA"

        # Turn on training mode for the model.
        self.set_mode(mode="training")

        # Reset best test auc
        self.best_val_auc = 0
        self.best_epoch = 0
        self.patient = 0

        
        # Start joint training of Autoencoder with Projection network
        for epoch in range(1,config['epochs']):
            
            # Keep a record of epoch
            self.epoch = epoch
            
            # 0 - Update Autoencoder
            self.update_autoencoder(train_data)
            
            # 1 - Update log message using epoch and batch numbers
           
            if epoch % 40 == 0:
                   self.update_log(epoch)
            
            # 2 - Clean-up for efficient memory usage.
            gc.collect()                
                
            # 3 - Run Validation
            self.run_validation(train_data, validation_data)

            # 4 - Change learning rate if scheduler==True
            _ = self.scheduler.step() if self.config["scheduler"] else None
            
            # 5 - Stop training if we run out of patience
            if self.patient == self.config["patience"]:
                break
            
        # Get the test performance and computing inference time 
       
        start = time.time()
        self.test_auc, self.test_ap = self.autoencoder.single_test(train_data, test_data)
        end = time.time()
        self.t_inference= end-start

        # Save plots of training loss and validation auc
        save_loss_plot(self.metrics, self._plots_path)
        save_auc_plot(self.metrics, self._plots_path)
        
        # Convert loss dictionary to a dataframe
        loss_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in self.metrics.items()]))
        
        # Save loss dataframe as csv file for later use
        loss_df.to_csv(self._loss_path + "/losses.csv")
        
            
    def run_validation(self, train_data: Union[List, torch.Tensor], validation_data: Union[List, torch.Tensor]) -> None:
        """Runs validation on the trained model and save weights if validation AUC improves.

        Parameters
        ----------
        train_data : List or torch.Tensor
            The training dataset.

        validation_data : List or torch.Tensor
            The validation dataset.
        """
        
        # Set the evaluation mode
        self.set_mode(mode="evaluation")
        
        # Validate every nth epoch. n=1 by default, but it can be changed in the config file
        if self.config["validate"]:

            # Compute validation AUCs
            self.val_auc, _ = self.autoencoder.single_test(train_data, validation_data)
                
            # Append auc's to the list to use for plots
            self.metrics["val_auc"].append(self.val_auc)
                
        # Save intermediate model on regular intervals
        if self.epoch >=self.config["nth_epoch"] and self.epoch % self.config["nth_epoch"] == 0:
            
            # Check the test auc at this epoch
            self.config["model_at_epoch"] = self.epoch
            val_auc, _ = self.autoencoder.single_test(train_data, validation_data)

            # Update the metrics.
            if val_auc > self.best_val_auc:
                self.best_val_auc = val_auc
                self.best_epoch = self.epoch 
                self.save_weights()
                self.patient = 0
            else:
                self.patient += 1
                    
        # Set training mode back
        self.set_mode(mode="training")
    

    def update_autoencoder(self, subgraphs: List[Data]) -> None:
        """Updates autoencoder model.

        Parameters
        ----------
        subgraphs : list of Data
            A list that contains subgraphs + original training graph.
        """
        total_loss, contrastive_loss, recon_loss, zrecon_loss = [], [], [], []

        # Last element of the list is the original whole training graph
        graph = subgraphs[-1]

        # If len(subgraphs) > 1, it means that we sampled subgraphs from the graph. Else, we have a standard GAE
        subgraphs = subgraphs[:-1] if len(subgraphs) > 1 else subgraphs

        # A list to hold list of latents --- will be used to compute contrastive loss
        z_list = []

        # Initialize total loss
        tloss = None

        # pass subgraphs through model to reconstruct the original graph from subgraphs
        for sg in subgraphs:

            # Reference graph
            ref_graph = graph if self.config["full_graph"] else sg

            # Drop edges if True
            if self.config["add_noise"]:
                sg.edge_index, sg.edge_attr = dropout_adj(sg.edge_index, edge_attr= sg.edge_attr, p=self.config["p_noise"])

            # Forwards pass
            z, latent = self.autoencoder(sg.x, sg.edge_index)           

            # Reconstruction loss by using GAE's native function
            rloss = self.autoencoder.gae.recon_loss(latent, ref_graph.pos_edge_label_index)

            # If the model is a variational model
            if self.autoencoder.variational:
                rloss = rloss + (1 / ref_graph.num_nodes) * self.autoencoder.gae.kl_loss()

            # Store z to the list
            z_list.append(z)

            # total loss
            tloss = tloss + rloss if tloss  is not None else rloss

            # Accumulate losses
            total_loss.append(tloss)
            recon_loss.append(rloss)

        # Compute the losses
        n = len(total_loss)
        total_loss = sum(total_loss) / n
        recon_loss = sum(recon_loss) / n

        # If the graph is large such as pubmed, push the losses to cpu.
        if self.config["dataset"] == "pubmed":
            total_loss = total_loss.cpu()
            recon_loss = recon_loss.cpu()

        # Initialize contrastive loss
        closs = None
        zloss = None

        if self.config["contrastive_loss"] and len(subgraphs)>1:

            # Generate combinations of z's to compute contrastive loss
            z_combinations = self.get_combinations_of_subgraphs(z_list)

            # Compute the contrastive loss for each pair of latent vectors
            for z in z_combinations:
                # Contrastive loss
                zloss = self.joint_loss(z)

                # Total contrastive loss
                closs = closs + zloss if closs is not None else zloss

            # Mean constrative loss
            closs = closs/len(z_combinations)

        # Update total loss
        total_loss = total_loss + closs if closs is not None else total_loss

        
        if self.is_reg :
            l2_reg_sum = torch.tensor(0., requires_grad=True).to(self.device)
            for param in self.autoencoder.parameters():
                l2_reg_sum = l2_reg_sum + torch.norm(param, 2)**2 
            total_loss = total_loss + self.config["l2_reg"] * l2_reg_sum
        # Add L2 regularization without in-place operation
        

        # Record losses
        self.metrics["tloss_e"].append(total_loss.item())
        self.metrics["rloss_e"].append(recon_loss.item())
        self.metrics["zloss_e"].append(closs.item() if closs is not None else 0)

        # Update Autoencoder params
        self._update_model(total_loss, self.optimizer_ae, retain_graph=True)

        # Delete loss and associated graph for efficient memory usage
        del total_loss, recon_loss, closs, zloss
        gc.collect()


    def get_combinations_of_subgraphs(self, z_list: List[Data]) -> List[Tuple[Data, Data]]:
        """Generates a list of combinations of subgraphs from the list of subgraphs.

        Parameters
        ----------
        z_list : list of Data
            List of subgraphs e.g. [z1, z2, z3, ...]

        Returns
        -------
        list of tuple
            A list of combinations of subgraphs e.g. [(z1, z2), (z1, z3), ...]
        """                            
        # Compute combinations of subgraphs [(z1, z2), (z1, z3)...]
        subgraph_combinations = list(itertools.combinations(z_list, 2))
        # List to store the concatenated subgraphs
        concatenated_subgraphs_list = []
        
        # Go through combinations
        for (zi, zj) in subgraph_combinations:
            # Concatenate xi, and xj, and turn it into a tensor
            z = torch.cat((zi, zj), dim=0)
            
            # Add it to the list
            concatenated_subgraphs_list.append(z)
        
        # Return the list of combination of subgraphs
        return concatenated_subgraphs_list
    
    def clean_up_memory(self, losses: List) -> None:
        """Deletes losses with attached graph, and cleans up memory.

        Parameters
        ----------
        losses : list
            List of loss values to be deleted.
        """
        for loss in losses: del loss
        gc.collect()

    def update_log(self, epoch: int) -> None:
        """Updates the messages displayed during training and evaluation.

        Parameters
        ----------
        epoch : int
            The current epoch number.
        """
        # For the first epoch, add losses for batches since we still don't have loss for the epoch
        if epoch < 1:
            description = f"Epoch:[{epoch - 1}], Total loss:{self.metrics['tloss_e'][-1]:.4f}"
            description += f", X recon loss:{self.metrics['rloss_e'][-1]:.4f}"
            if self.config["contrastive_loss"]:
                description += f", contrastive loss:{self.metrics['zloss_e'][-1]:.6f}"
            description += f", val auc:{self.val_auc}"

        # For sub-sequent epochs, display only epoch losses.
        else:
            description = f"Epoch:[{epoch - 1}] training loss:{self.metrics['tloss_e'][-1]:.4f}"
            description += f", X recon loss:{self.metrics['rloss_e'][-1]:.4f}"
            if self.config["contrastive_loss"]:
                description += f", contrastive loss:{self.metrics['zloss_e'][-1]:.6f}"
            # Add validation auc
            description += f", val auc:{self.val_auc}"

        # Update the displayed message
        print(description)

    def set_mode(self, mode: str = "training") -> None:
        """Sets the mode of the models, either as .train(), or .eval().

        Parameters
        ----------
        mode : str, optional
            Mode in which to set the models, by default "training".
        """
        for _, model in self.model_dict.items():
            model.train() if mode == "training" else model.eval()

    
    
    def save_weights(self,is_pruned:bool=False, with_epoch: bool = False) -> None:
        """Saves weights of the models.

        Parameters
        ----------
        with_epoch : bool, optional
            If True, includes the epoch number in the filename, by default False.
        """
        for model_name in self.model_dict:
            
            # Check if we want to save the model at a specific epoch
            file_name = model_name + "_" + str(self.epoch) if with_epoch else model_name
            
            # Save the model
            if self.is_reg==False:
            
                dill.dump(self.model_dict[model_name], open(self._model_path + "/" + file_name + ".pt", 'wb'))
            if self.is_reg==True:  
                dill.dump(self.model_dict[model_name], open(self._model_path + "/" + file_name + "reg.pt", 'wb'))
        
        print("Done with saving models.")

   

    def load_models(self, epoch: Optional[int] = None) -> None:
        """Loads weights saved at the end of the training.

        Parameters
        ----------
        epoch : int, optional
            If provided, loads the weights saved at the specified epoch, by default None.
        """
        for model_name in self.model_dict:
            
            # Check if we want to load the model saved at a specific epoch
            file_name = model_name + "_" + str(epoch) if epoch is not None else model_name

            # Load the model
            if self.is_reg==False:
                model = dill.load(open(self._model_path + "/" + file_name + ".pt", 'rb'))
            if self.is_reg==True:
                
                model = dill.load(open(self._model_path + "/" + file_name + "reg.pt"), 'rb')
            # Register model to the class
            setattr(self, model_name, model.eval())
            print(f"--{model_name} is loaded")
        
        print("Done with loading models.")
    
    
    
    def print_model_summary(self) -> None:
        """Displays model architectures as a sanity check to see if the models are constructed correctly."""
        # Summary of the model
        description = f"{40 * '-'}Summary of the models:{40 * '-'}\n"
        description += f"{34 * '='} NESS Architecture {34 * '='}\n"
        description += f"{self.autoencoder}\n"
        # Print model architecture
        print(description)

    def _update_model(self, loss: torch.Tensor, optimizer: torch.optim.Optimizer, retain_graph: bool = True) -> None:
        """Does backpropagation and updates the model parameters.

        Parameters
        ----------
        loss : torch.Tensor
            Loss containing computational graph.

        optimizer : torch.optim.Optimizer
            Optimizer used during training.

        retain_graph : bool, optional
            If True, retains the computational graph after backpropagation, by default True.
        """
        # Reset optimizer
        optimizer.zero_grad()
        # Backward propagation to compute gradients
        loss.backward(retain_graph=retain_graph)
        # Update weights
        optimizer.step()

    def _set_scheduler(self) -> None:
        """Sets a scheduler for the learning rate of the autoencoder."""
        # Set scheduler (Its use will be optional)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer_ae, step_size=1, gamma=0.99)

    def _set_paths(self) -> None:
        """Sets paths to be used for saving results at the end of the training."""
        # Top results directory
        self._results_path = os.path.join(self.config["paths"]["results"], self.config["experiment"])
        # Directory to save model
        self._model_path = os.path.join(self._results_path, "training", "model")
        # Directory to save plots as png files
        self._plots_path = os.path.join(self._results_path, "training", "plots")
        # Directory to save losses as csv file
        self._loss_path = os.path.join(self._results_path, "training", "loss")

    def _adam(self, params: Union[List, Tuple], lr: float = 1e-4,weight_decay: float = 0) -> torch.optim.AdamW:
        """Sets up AdamW optimizer using model parameters.

        Parameters
        ----------
        params : list or tuple
            Parameters of the models to optimize.

        lr : float, optional
            Learning rate, by default 1e-4.

        weight_decay: float, optional
            L2 regularization coefficient, by default 1e-5.

        Returns
        -------
        torch.optim.AdamW
            AdamW optimizer.
         """
        return torch.optim.AdamW(itertools.chain(*params), lr=lr, betas=(0.9, 0.999), eps=1e-07, weight_decay= 0)
       

### Function for training dataset

In [9]:
# We need these two function for training dataset

def train(config: Dict, data_loader: IterableDataset, save_weights: bool = True) -> None:
    """
    Trains the model using provided configuration and data loader.

    Parameters
    ----------
    config : dict
        Dictionary containing options.

    data_loader : IterableDataset
        Pytorch data loader used for training the model.

    save_weights : bool, optional
        If True, the trained model is saved. By default, it's True.
    
    """
    
    # Instantiate model
    model = NESS(config)
    # Start the clock to measure the training time
    start = time.process_time()
    # Fit the model to the data
    model.fit(data_loader)
    # Total time spent on training
    training_time = time.process_time() - start
    # Report the training time
    print("Done with training...")
    print(f"Training time:  {training_time//60} minutes, {training_time%60} seconds")
    

    
    # Return the best Test set AUC
    return model.test_auc, model.test_ap, model.t_inference, model.val_auc, model


def main(config: Dict) -> None:
    """
    The main function that starts the execution of the program. Takes the 
    configuration dictionary as input.

    Parameters
    ----------
    config : dict
        Dictionary containing options.
    """
   
    # Ser directories (or create if they don't exist)
    set_dirs(config)
    # Get data loader for first dataset.
    ds_loader = GraphLoader(config, dataset_name=config["dataset"])
    # Add the number of features in a dataset as the first dimension of the model
    config = update_config_with_model_dims(ds_loader, config)
    # Start training and save model weights at the end
    test_auc, test_ap,t_inference,val_auc, model= train(config, ds_loader, save_weights=True)
    # Return best test auc
    return test_auc, test_ap, t_inference, val_auc, model

##  Measurement 

#### Setting Argument

In [10]:
# Get parser / command line arguments
args = get_arguments()
# Get configuration file
config = get_config(args)


# By default, we are using the name of the dataset. This can be customized.
config["experiment"] = config["dataset"]

# File name to use when saving results as csv. This can be customized
config["file_name"] = config["experiment"] + "_sub" + str(config["n_subgraphs"]) + '_seed' + str(config["seed"])

# The model path
results_path = os.path.join(config["paths"]["results"], config["experiment"])

Device being used is cpu


#### Setting Regularization rate

In [11]:

### Regularization Rate
l2_lambda = 10000


#The number of epochs 
config["epochs"]=100
# The number of iteration
num_iterations=10

In [12]:

# This is a dictionary to save all measurements. Aftre measuring, we can compute mean and std of each item.
Eva_final=dict()

# The following are all list of criteria for measurements. 
# We collect all desired datas of each list across iterations. 
# Then, we compute average and std of each list.

#Base model
Base_model_accuracy=[]
T_base_model=[]
Num_parm_base_model=[]
Base_model_size=[]
Base_Energy_Consumption=[]
Base_Cpu_Usage=[]
Base_Memory_Usage=[]

#regularized model
Reg_model_accuracy=[]
T_Reg_model=[]
Num_parm_Reg_model=[]
Reg_model_size=[]
Reg_Energy_Consumption=[]
Reg_Cpu_Usage=[]
Reg_Memory_Usage=[]

# Here is the dictionary to record the list of all measurements
Eva_measure={'base model accuracy':Base_model_accuracy,
            'time inference of base model':T_base_model,
            'number parmameters of base model':Num_parm_base_model,
            'base model size':Base_model_size,
            'energy consumption of base model':Base_Energy_Consumption,
            'cpu usage of base model':Base_Cpu_Usage,
            'memory usage of base model':Base_Memory_Usage,
            'regularized model accuracy': Reg_model_accuracy,
            'time inference of regularized model':T_Reg_model,
            'number parmameters of regularized model':Num_parm_Reg_model,
            'regularized model size':Reg_model_size,
            'energy consumption of regularized model':Reg_Energy_Consumption,
            'cpu usage of regularized model':Reg_Cpu_Usage,
            'memory usage of regularized model':Reg_Memory_Usage
            }

             


### Trainig and regularization

In [ ]:
ds_loader = GraphLoader(config, dataset_name=config["dataset"])
train_data = ds_loader.train_data
validation_data = ds_loader.validation_data
test_data = ds_loader.test_data



for i in range(num_iterations):
        print('________________________________________________')
        print('************************************************')
        print(f"This is iteration :{i+1}")
        
        print(f'Training and evaluation before regularization ')
        print("Starting training...")


        Eva=dict() 
        config["is_reg"]=False
        config["l2_reg"]=0
        _, _, _,_, model = main(config)

        best_checkpoint = dict()
        best_checkpoint['state_dict'] = copy.deepcopy(model.autoencoder.state_dict())
        model.autoencoder.load_state_dict(best_checkpoint['state_dict'])
        recover_model = lambda: model.autoencoder.load_state_dict(best_checkpoint['state_dict'])



        base_model_path = os.path.join(results_path, "training", "model", "autoencoder.pt")


        # Start monitoring CPU and memory usage, model size, number of parametes, time inference and  power consumption
        gc.collect()
        time.sleep(5)  # Add a 5-second delay to stabilize the initial state
        tracemalloc.start()  # Start tracking memory allocations
        snapshot_before = tracemalloc.take_snapshot()#take a snapshot of the current memory state before starting the measurement.

        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)

        base_model_accuracy, test_ap=model.autoencoder.single_test(train_data, test_data)


        base_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_base_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        base_total_memory_diff = sum([stat.size_diff for stat in top_stats])

        base_energy_consumption = power_usage * t_base_model
        # model size
        base_model_size = os.path.getsize(base_model_path)
        # number of parameters
        num_parm_base_model=get_num_parameters(model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5) 

        print(f'*****Results of base model*********')

        print(f"base model has accuracy on test set={base_model_accuracy:.2f}%")
        print(f"base model has size={base_model_size:.2f} bytes")
        print(f"The time inference of base model is ={t_base_model}") 
        print(f"The number of parametrs of base model is:{num_parm_base_model}") 

        print(f"Energy Consumption : {base_energy_consumption:.3f}")
        print(f"total memory usage of base model':{base_total_memory_diff} ")
        print(f"cpu usage of base model':{base_cpu_usage:.3f} %")


        #Update Eva dictionary
        Eva.update({'base model accuracy': base_model_accuracy,
                    'time inference of base model': t_base_model,
                    'number parmameters of base model': num_parm_base_model,
                    'size of base model': base_model_size, 
                    'energy consumption of base model':base_energy_consumption,
                    'total memory usage of base model':base_total_memory_diff,
                    'cpu usage of base model':base_cpu_usage
                   })

        gc.collect()
        time.sleep(5)  

        #### Regularization of the Model
        gc.collect()
        time.sleep(5)   



        print('________*******************************_____________')
        print(f'Regularized Model')

        config["is_reg"]=True
        config["l2_reg"]= l2_lambda
        _, _, _,_, model=main(config)

        #### load the best regularized model
        reg_model_path =  os.path.join(results_path, "training", "model","autoencoder" + "reg.pt")
        best_checkpoint['state_dict'] = copy.deepcopy(model.autoencoder.state_dict())
        model.autoencoder.load_state_dict(best_checkpoint['state_dict'])
        recover_model = lambda: model.autoencoder.load_state_dict(best_checkpoint['state_dict'])



        # Result of regularization


        gc.collect()
        time.sleep(5)  
        tracemalloc.start() 
        snapshot_before = tracemalloc.take_snapshot()

        t0 = time.perf_counter()
        initial_cpu_usage = get_cpu_usage()
        power_usage = estimate_power_usage(initial_cpu_usage)

        regularized_model_accuracy, regularized_test_ap=model.autoencoder.single_test(train_data, test_data)


        regularized_cpu_usage = get_cpu_usage()
        t1 = time.perf_counter()
        t_regularized_model=t1-t0

        snapshot_after = tracemalloc.take_snapshot()
        tracemalloc.stop()
        top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

        regularized_total_memory_diff = sum([stat.size_diff for stat in top_stats])
        regularized_energy_consumption = power_usage * t_regularized_model
        regularized_model_size = os.path.getsize( reg_model_path )
        num_parm_regularized_model=get_num_parameters(model, count_nonzero_only=True)

        gc.collect()
        time.sleep(5)  # Add a 5-second delay to stabilize the initial state    



        print('****************Result of regularized model ******************')


        print(f"{l2_lambda} regularized model has accuracy on test set={regularized_model_accuracy:.2f}%")
        print(f"{l2_lambda} regularized model has size={regularized_model_size:.2f} bytes")
        print(f"The time inference of {l2_lambda} regularized model is ={t_regularized_model}") 
        print(f"The number of parametrs of {l2_lambda} regularized model is:{num_parm_regularized_model}") 

        print(f"Energy Consumption of {l2_lambda} regularized model: {regularized_energy_consumption:.3f}")
        print(f"total memory usage of {l2_lambda} regularized model':{regularized_total_memory_diff} ")
        print(f"cpu usage of {l2_lambda} regularized model':{regularized_cpu_usage:.3f} %")


        #Update Eva dictionary
        Eva.update({'regularized model accuracy': regularized_model_accuracy,
                    'time inference of regularized model': t_regularized_model,
                    'number parmameters of regularized model': num_parm_regularized_model,
                    'size of regularized model': regularized_model_size, 
                    'energy consumption of regularized model':regularized_energy_consumption,
                    'total memory usage of regularized model':regularized_total_memory_diff,
                    'cpu usage of regularized model':regularized_cpu_usage
                   })

        gc.collect()
        time.sleep(5)   



        Base_model_accuracy.append(Eva['base model accuracy'])
        T_base_model.append(Eva['time inference of base model'])
        Num_parm_base_model.append(int(Eva['number parmameters of base model']))
        Base_model_size.append(int(Eva['size of base model']))
        Base_Energy_Consumption.append(Eva['energy consumption of base model'])
        Base_Cpu_Usage.append(Eva['cpu usage of base model'])
        Base_Memory_Usage.append(Eva['total memory usage of base model'])

        Reg_model_accuracy.append(Eva['regularized model accuracy'])
        T_Reg_model.append(Eva['time inference of regularized model'])
        Num_parm_Reg_model.append(int(Eva['number parmameters of regularized model']))
        Reg_model_size.append(int(Eva['size of regularized model']))
        Reg_Energy_Consumption.append(Eva['energy consumption of regularized model'])
        Reg_Cpu_Usage.append(Eva['cpu usage of regularized model'])
        Reg_Memory_Usage.append(Eva['total memory usage of regularized model'])




________________________________________________
************************************************
This is iteration :1
Training and evaluation before regularization 
Starting training...
Directories are set.
 Is the approach is regularization:False
Regrularization rate is:0
Directories are set.
Building the models for training and evaluation in NESS framework...
----------------------------------------Summary of the models:----------------------------------------
================================== NESS Architecture ==================================
GAEWrapper(
  (gae): GAE(
    (encoder): GNAEEncoder(
      (linear1): Linear(in_features=1433, out_features=32, bias=True)
      (propagate): APPNP(K=1, alpha=0)
    )
    (decoder): InnerProductDecoder()
  )
)

Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[39] training loss:2.3815, X recon loss:0.9655, val auc:0.8971360002313173
Done with saving models.
Done with saving models.
Done with saving models.


________________________________________________
************************************************
This is iteration :4
Training and evaluation before regularization 
Starting training...
Directories are set.
 Is the approach is regularization:False
Regrularization rate is:0
Directories are set.
Building the models for training and evaluation in NESS framework...
----------------------------------------Summary of the models:----------------------------------------
================================== NESS Architecture ==================================
GAEWrapper(
  (gae): GAE(
    (encoder): GNAEEncoder(
      (linear1): Linear(in_features=1433, out_features=32, bias=True)
      (propagate): APPNP(K=1, alpha=0)
    )
    (decoder): InnerProductDecoder()
  )
)

Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[39] training loss:2.3815, X recon loss:0.9655, val auc:0.8971360002313173
Done with saving models.
Done with saving models.
Done with saving models.


### Computing Mean and Std of Lists

In [29]:
Eva_final=dict()
base_model_accuracy_mean = stat.mean(Base_model_accuracy)
base_model_accuracy_std =  stat.stdev(Base_model_accuracy)
Eva_final.update({'Ave of base model accuracy':float(format(base_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of base model accuracy':float(format(base_model_accuracy_std, '.3f'))})
base_model_accuracy = "{:.3f} ± {:.3f}".format(base_model_accuracy_mean ,base_model_accuracy_std)
print(f"Base model accuracy is:{base_model_accuracy}")

                 
t_base_model_mean =stat.mean(T_base_model)
t_base_model_std =stat.stdev(T_base_model)  
Eva_final.update({'Ave of time inference of base model':float(format(t_base_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of base model':float(format(t_base_model_std, '.3f'))})
t_base_model = "{:.3f} ± {:.3f}".format(t_base_model_mean ,t_base_model_std)
print(f"Time inference of Base model :{t_base_model}")


num_parm_base_model_mean = stat.mean(Num_parm_base_model)
num_parm_base_model_std = stat.stdev(Num_parm_base_model)
Eva_final.update({'Ave of number parmameters of base model':num_parm_base_model_mean})
Eva_final.update({'Std of number parmameters of base model':num_parm_base_model_std})
num_parm_base_model = "{:.3f} ± {:.3f}".format(num_parm_base_model_mean ,num_parm_base_model_std)
print(f"Time number of parameters of Base model :{num_parm_base_model}")

base_model_size_mean = stat.mean(Base_model_size)
base_model_size_std = stat.stdev(Base_model_size)
Eva_final.update({'Ave of base model size':base_model_size_mean})
Eva_final.update({'Std of base model size':base_model_size_std})
base_model_size_model = "{:.3f} ± {:.3f}".format(base_model_size_mean ,base_model_size_std)
print(f"The size of Base model :{base_model_size} bytes")


base_energy_consumption_mean = stat.mean(Base_Energy_Consumption)
base_energy_consumption_std = stat.stdev(Base_Energy_Consumption)
Eva_final.update({'Ave of energy consumption of base model':base_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of base model':base_energy_consumption_std})
base_energy_consumption = "{:.3f} ± {:.3f}".format(base_energy_consumption_mean ,base_energy_consumption_std)
print(f"The energy consumption of Base model :{base_energy_consumption} ")


base_cpu_usage_mean = stat.mean(Base_Cpu_Usage)
base_cpu_usage_std = stat.stdev(Base_Cpu_Usage)
Eva_final.update({'Ave of cpu usage of base model':base_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of base model':base_cpu_usage_std})
base_cpu_usage = "{:.3f} ± {:.3f}".format(base_cpu_usage_mean ,base_cpu_usage_std)
print(f"The CPU usage of Base model :{base_cpu_usage} ")


base_memory_usage_mean = stat.mean(Base_Memory_Usage)
base_memory_usage_std = stat.stdev(Base_Memory_Usage)
Eva_final.update({'Ave of memory usage of base model':base_memory_usage_mean})
Eva_final.update({'Std of memory usage of base model':base_memory_usage_std})
base_memory_usage = "{:.3f} ± {:.3f}".format(base_memory_usage_mean ,base_memory_usage_std)
print(f"The memory usage of Base model :{base_memory_usage} ")

print(100 * "=")
####################################################

reg_model_accuracy_mean =stat.mean(Reg_model_accuracy)
reg_model_accuracy_std = stat.stdev(Reg_model_accuracy)
Eva_final.update({'Ave of regularized model accuracy':float(format(reg_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of regularized model accuracy':float(format(reg_model_accuracy_std, '.3f'))})
reg_model_accuracy = "{:.3f} ± {:.3f}".format(reg_model_accuracy_mean ,reg_model_accuracy_std)
print(f"Regularized model accuracy is:{reg_model_accuracy}")
                 

t_reg_model_mean = stat.mean(T_Reg_model)
t_reg_model_std =stat.stdev(T_Reg_model)
Eva_final.update({'Ave of time inference of regularized model':float(format(t_reg_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of regularized model':float(format(t_reg_model_std, '.3f'))})
t_reg_model = "{:.3f} ± {:.3f}".format(t_reg_model_mean ,t_reg_model_std)
print(f"Time inference of Regularized model :{t_reg_model}")

num_parm_reg_model_mean = stat.mean(Num_parm_Reg_model)
num_parm_reg_model_std = stat.stdev(Num_parm_Reg_model)
Eva_final.update({'Ave of number parmameters of regularized model':num_parm_reg_model_mean})
Eva_final.update({'Std of number parmameters of regularized model':num_parm_reg_model_std})
num_parm_reg_model = "{:.3f} ± {:.3f}".format(num_parm_reg_model_mean ,num_parm_reg_model_std)
print(f"Time number of parameters of Regularized model :{num_parm_reg_model}")

reg_model_size_mean =stat.mean( Reg_model_size)
reg_model_size_std = stat.stdev(Reg_model_size)
Eva_final.update({'Ave of regularized model size':reg_model_size_mean})
Eva_final.update({'Std of regularized model size':reg_model_size_std })
reg_model_size = "{:.3f} ± {:.3f}".format(reg_model_size_mean ,reg_model_size_std)
print(f"The size of Regularized model :{reg_model_size} bytes")

reg_energy_consumption_mean = stat.mean(Reg_Energy_Consumption)
reg_energy_consumption_std = stat.stdev(Reg_Energy_Consumption)
Eva_final.update({'Ave of energy consumption of regularized model':reg_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of regularized model':reg_energy_consumption_std})
reg_energy_consumption = "{:.3f} ± {:.3f}".format(reg_energy_consumption_mean ,reg_energy_consumption_std)
print(f"The energy consumption of Regularized model :{reg_energy_consumption} ")


reg_cpu_usage_mean = stat.mean(Reg_Cpu_Usage)
reg_cpu_usage_std = stat.stdev(Reg_Cpu_Usage)
Eva_final.update({'Ave of cpu usage of regularized model':reg_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of regularized model':reg_cpu_usage_std})
reg_cpu_usage = "{:.3f} ± {:.3f}".format(reg_cpu_usage_mean ,reg_cpu_usage_std)
print(f"The CPU usage of Regularized model :{reg_cpu_usage} ")


reg_memory_usage_mean = stat.mean(Reg_Memory_Usage)
reg_memory_usage_std = stat.stdev(Reg_Memory_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of memory usage of regularized model':reg_memory_usage_mean})
Eva_final.update({'Std of memory usage of regularized model':reg_memory_usage_std})
reg_memory_usage = "{:.3f} ± {:.3f}".format(reg_memory_usage_mean ,reg_memory_usage_std)
print(f"The memory usage of Regularized model :{reg_memory_usage} ")



#################################

print(100 * "=")
print(f"All measurement about regularization process of rate:{l2_lambda} ")   
Eva_final

Base model accuracy is:0.909 ± 0.016
Time inference of Base model :2.098 ± 0.018
Time number of parameters of Base model :45888.000 ± 0.000
The size of Base model :187684 bytes
The energy consumption of Base model :69.834 ± 30.260 
The CPU usage of Base model :45.560 ± 30.101 
The memory usage of Base model :16643.000 ± 1392.893 
Regularized model accuracy is:0.800 ± 0.119
Time inference of Regularized model :2.135 ± 0.110
Time number of parameters of Regularized model :45888.000 ± 0.000
The size of Regularized model :187688.600 ± 3.098 bytes
The energy consumption of Regularized model :72.549 ± 40.796 
The CPU usage of Regularized model :47.850 ± 34.898 
The memory usage of Regularized model :16166.300 ± 68.767 
All measurement about regularization process of rate:10000 


{'Ave of base model accuracy': 0.909,
 'Std of base model accuracy': 0.016,
 'Ave of time inference of base model': 2.098,
 'Std of time inference of base model': 0.018,
 'Ave of number parmameters of base model': 45888,
 'Std of number parmameters of base model': 0.0,
 'Ave of base model size': 187686.8,
 'Std of base model size': 3.6147844564602556,
 'Ave of energy consumption of base model': 69.83412391841993,
 'Std of energy consumption of base model': 30.26036674807619,
 'Ave of cpu usage of base model': 45.56,
 'Std of cpu usage of base model': 30.10069028370538,
 'Ave of memory usage of base model': 16643,
 'Std of memory usage of base model': 1392.892833079575,
 'Ave of regularized model accuracy': 0.8,
 'Std of regularized model accuracy': 0.119,
 'Ave of time inference of regularized model': 2.135,
 'Std of time inference of regularized model': 0.11,
 'Ave of number parmameters of regularized model': 45888,
 'Std of number parmameters of regularized model': 0.0,
 'Ave of regu

### Recording results in txt file

In [30]:

dataset_name = 'Cora-Link'
Pruning_Method='Regularization'
max_epoch = 100
resume = True
result_folder ='pathresult/'
if not os.path.exists(result_folder):
    os.makedirs(result_folder)



file_name = result_folder+Pruning_Method+'_'+'with rate of regularization of'+'_'+str(l2_lambda)+'_on_'+dataset_name+'_'+str(max_epoch)+'.txt'
with open(file_name, 'w') as f:
        f.write('%s:%s\n'%('dataset_name', 'Cora-Link'))
        f.write('%s:%s\n'%('max_epoch', max_epoch))
        f.write('%s:%s\n'%('sparsity', l2_lambda))
        
        for key, value in Eva_final.items():
            f.write('%s:%s\n'%(key, value))
  
        for key, value in Eva_measure.items():
            f.write('%s:%s\n' % (key, ','.join(map(str, value)))) 
       